In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import pandas as pd

In [3]:
data_path = "/weka/scratch/weka/kellislab/rcalef/data/"

# Core types overview

The core datasets used in this repo pertain to protein substructures, i.e. small subsets of a protein's overall 3D structure that are evolutionarily conserved and observed across many distinct proteins.  These sub-structures form the lower-level building blocks that make up the 3D strucutre of a full-length protein, and some also have distinct functions.

We use two main sources for our substructure annotations:
- InterPro: "provides functional analysis of proteins by classifying them into families and predicting domains and important sites". InterPro annotations include functional domains, binding sites, and higher-order structural groupings like protein families.
- DSSP: provides annotation of secondary structure elements given an experimental or predicted protein structure

In this notebook, we explore the representation of these datatypes in our current codebase.

The core types are defined in `magneton/types.py`, the most notable of which is the `Protein` class.

```
@dataclass
class Protein:
    uniprot_id: str
    kb_id: str
    name: str
    length: int
    parsed_entries: int
    total_entries: int
    entries: List[InterproEntry]
    secondary_structs: List[SecondaryStructure] = field(default_factory=list)
```

This class represents a single protein, specified by a UniProt ID, and provides its InterPro annotations and secondary structure predictions.

These `Protein` objects are currently stored as Python pickle files.

In [4]:
from magneton.io.internal import parse_from_pkl

In [5]:
pkl_path = os.path.join(
    data_path,
    "interpro",
    "103.0",
    "swissprot",
    "sharded_swissprot",
    "with_ss",
    "swissprot.with_ss.0.pkl.bz2",
)

example_prots = []
for i, prot in enumerate(parse_from_pkl(pkl_path)):
    example_prots.append(prot)
    if i == 10:
        break

# The `print()` method just pretty prints the object.
example_prots[0].print()

Protein(uniprot_id='A0A009IHW8',
        kb_id='sp|A0A009IHW8|ABTIR_ACIB9',
        name='ABTIR_ACIB9',
        length=269,
        parsed_entries=5,
        total_entries=5,
        entries=[InterproEntry(id='IPR035897',
                               element_type='Homologous_superfamily',
                               match_id='G3DSA:3.40.50.10140',
                               element_name='Toll/interleukin-1 receptor '
                                            'homology (TIR) domain superfamily',
                               representative=False,
                               positions=[(80, 266)]),
                 InterproEntry(id='IPR000157',
                               element_type='Domain',
                               match_id='PF13676',
                               element_name='Toll/interleukin-1 receptor '
                                            'homology (TIR) domain',
                               representative=False,
                               p

In the above example, `parse_from_pkl` returns an iterator that reads `Protein` objects from the pickle file one at a time.

Each `Protein` object contains the corresponding protein's InterPro annotations as `InterproEntry` objects:
```
@dataclass
class InterproEntry:
    id: str
    element_type: str
    match_id: str
    element_name: str
    representative: bool
    # Note that positions are 1-indexed, i.e. exactly as given in InterPro.
    positions: List[Tuple[int]]
```
These objects specify the unique InterPro ID for this specific substructure annotation, what type of annotation it is (e.g. Domain, Family, etc.), and where it occurs in the given protein. Note that since these substructures occurr in 3D, they can be made up of amino acids that are not adjacent in the underlying amino acid sequence, hence being represented as a list of ranges.

In [6]:
example_prots[0].entries[0].print()

InterproEntry(id='IPR035897',
              element_type='Homologous_superfamily',
              match_id='G3DSA:3.40.50.10140',
              element_name='Toll/interleukin-1 receptor homology (TIR) domain '
                           'superfamily',
              representative=False,
              positions=[(80, 266)])


We also provide the secondary structure annotations as `SecondaryStructure` object, which just provides the type of secondary structure as an enum, and the contiguous range on the protein sequence (all secondary structures are contiguous):
```
@unique
class DsspType(Enum):
    H = 0
    B = auto()
    E = auto()
    G = auto()
    I = auto()
    P = auto()
    T = auto()
    S = auto()
    # In place of ' ' (space) for OTHER
    X = auto()

@dataclass
class SecondaryStructure:
    dssp_type: DsspType
    # Note that positions are 1-indexed, as output by dssp.
    # Coordinates are half-open, i.e. [start, end)
    start: int
    end: int
```

In [7]:
example_prots[0].secondary_structs[0].print()

Alphahelix: 3 - 21


# Dataset interface and implementation

Our overarching dataset is UniProtKB, which contains over 200 million proteins. To work with a smaller dataset, we use the manually reviewed subset of UniProtKB, which is referred to as SwissProt and contains ~550k proteins. However, since this can still be a large and unwieldy dataset to store as a single file, we store the dataset as many files, i.e. as a "sharded" dataset. 

This means that we can choose to either process the dataset as a whole by reading all the shards in sequence, or can trivially parallelize any operation that operates on single `Protein`s by parallelizing over the many shards.

To abstract whether the underlying dataset is a single file or a directory of sharded files, we introduce the `ProteinDataset` object. Here we read a small number of proteins from a sharded directory.

In [8]:
from magneton.io.internal import ProteinDataset

In [9]:
prot_dataset = ProteinDataset(
    os.path.join(
        data_path,
        "interpro",
        "103.0",
        "swissprot",
        "sharded_swissprot",
        "with_ss"
    ),
    # Provide the prefix for the sharded files, named like
    # {prefix}.{shard_num}.pkl.bz2
    prefix="swissprot.with_ss",
)

In [10]:
# These shards each contain 10000 proteins, so we'll demonstrate
# reading from multiple shards seamlessly.
example_prots = []
for i, prot in enumerate(prot_dataset):
    if i == 15000:
        break
    example_prots.append(prot)

len(example_prots)

15000

To speed up fetching a single protein from the dataset, we also use an index to identify which shard a protein is contained in, and can then fetch just from that shard.

In [11]:
prot_dataset.fetch_protein("B1H267").print()

Protein(uniprot_id='B1H267',
        kb_id='sp|B1H267|SNX5_RAT',
        name='SNX5_RAT',
        length=404,
        parsed_entries=10,
        total_entries=11,
        entries=[InterproEntry(id='IPR027267',
                               element_type='Homologous_superfamily',
                               match_id='G3DSA:1.20.1270.60',
                               element_name='AH/BAR domain superfamily',
                               representative=False,
                               positions=[(201, 403)]),
                 InterproEntry(id='IPR036871',
                               element_type='Homologous_superfamily',
                               match_id='G3DSA:3.30.1520.10',
                               element_name='PX domain superfamily',
                               representative=False,
                               positions=[(19, 180)]),
                 InterproEntry(id='IPR001683',
                               element_type='Domain',
                   

We can also leverage the sharded structure to quickly perform computations or subsets of the full dataset. 

In this example, we use the `filter_proteins` function that takes in a function that receives a `Protein` and returns a `bool`, and fetches all proteins that match that criteria. In particular, we pull out proteins with very long alpha helices.

Note that this example requires you to be running on a machine with multiple cores, set the `nprocs` parameter accordingly.

In [12]:
from magneton.types import DsspType, Protein
from magneton.io.internal import filter_proteins

In [13]:
def has_long_helix(prot: Protein, max_len: int = 500) -> bool:
    for ss in prot.secondary_structs:
        if ss.dssp_type == DsspType.H and (ss.end - ss.start) >= max_len:
            return True
    return False

long_helices = filter_proteins(
    prot_dataset.input_path,
    has_long_helix,
    prefix=prot_dataset.prefix,
    nprocs=32,
)
len(long_helices)
long_helices[0].print()

100%|████████████████████████████████████████████████████████████████████| 55/55 [00:07<00:00,  7.66it/s]


Protein(uniprot_id='A0A166B1A6',
        kb_id='sp|A0A166B1A6|NMCP1_DAUCS',
        name='NMCP1_DAUCS',
        length=1119,
        parsed_entries=1,
        total_entries=1,
        entries=[InterproEntry(id='IPR040418',
                               element_type='Family',
                               match_id='PTHR31908',
                               element_name='Protein crowded nuclei',
                               representative=True,
                               positions=[(3, 1119)])],
        secondary_structs=[SecondaryStructure(dssp_type=<DsspType.S: 7>,
                                              start=5,
                                              end=6),
                           SecondaryStructure(dssp_type=<DsspType.S: 7>,
                                              start=7,
                                              end=10),
                           SecondaryStructure(dssp_type=<DsspType.H: 0>,
                                              start=10

# Associated metadata

## All InterPro types/labels

If on OpenMind, an exhaustive list of the set of InterPro entries can be found here:

In [14]:
label_path = os.path.join(
    data_path,
    "interpro",
    "103.0",
    "label_sets",
)

def load_stats(interpro_type: str) -> pd.DataFrame:
    labels_path = os.path.join(label_path, f"{interpro_type}.labels.tsv")
    return pd.read_table(labels_path)

In [15]:
load_stats("Domain").head()

,label,element_name,interpro_id
0,0,Kringle,IPR000001
1,1,"Tubby, C-terminal",IPR000007
2,2,C2 domain,IPR000008
3,3,Cystatin domain,IPR000010
4,4,PAS domain,IPR000014


In [16]:
load_stats("Family").head()

,label,element_name,interpro_id
0,0,Retinoid X receptor/HNF4,IPR000003
1,1,"Metallothionein, vertebrate",IPR000006
2,2,Protein phosphatase 2A regulatory subunit PR55,IPR000009
3,3,Ubiquitin/SUMO-activating enzyme E1-like,IPR000011
4,4,Retroviral VpR/VpX protein,IPR000012


## Structures and sequences

For those working on OpenMind, you can find the associated 3D structure files from AlphaFoldDB here:

In [17]:
example_id = example_prots[0].uniprot_id
cif_tmpl = "/weka/scratch/weka/kellislab/rcalef/data/cif_alphafolddb/AF-%s-F1-model_v4.cif.gz"
pdb_tmpl = "/weka/scratch/weka/kellislab/rcalef/data/pdb_alphafolddb/AF-%s-F1-model_v4.pdb"

print(os.path.exists(cif_tmpl % example_id))
print(os.path.exists(pdb_tmpl % example_id))

True
True


Similarly, the associated amino acid sequence can be obtained as follows:

In [18]:
from pysam import FastaFile

In [19]:
fasta_path = "/weka/scratch/weka/kellislab/rcalef/data/uniprot/uniprot_sprot.fasta.bgz"

fa = FastaFile(fasta_path)

# Note the use of kb_id instead of uniprot_id
print(fa[example_prots[0].kb_id])

MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENARIQSKLSDLQKKKIDIDNKLLKEKQNLIKEEILERKKLEVLTKKQQKDEIEHQKKLKREIDAIKASTQYITDVSISSYNNTIPETEPEYDLFISHASEDKEDFVRPLAETLQQLGVNVWYDEFTLKVGDSLRQKIDSGLRNSKYGTVVLSTDFIKKDWTNYELDGLVAREMNGHKMILPIWHKITKNDVLDYSPNLADKVALNTSVNSIEEIAHQLADVILNR


## Dataset splits

Dataset splits for the SwissProt subset can be found on OpenMind at the following path, and are also structured as directories of sharded pickle files

In [20]:
splits_path = os.path.join(
    data_path,
    "interpro",
    "103.0",
    "swissprot",
    "sharded_swissprot",
    "with_ss",
    "dataset_splits",
)
print(splits_path)

/weka/scratch/weka/kellislab/rcalef/data/interpro/103.0/swissprot/sharded_swissprot/with_ss/dataset_splits


In [21]:
os.listdir(splits_path)

['dataset_splits.tsv', 'test_sharded', 'train_sharded', 'val_sharded']

For more details on split construction, see [`create_dataset_splits.ipynb`](https://github.com/rcalef/magneton/blob/main/experiments/create_dataset_splits.ipynb)